# Fairness Toolkit Demo (v0.1.0)

This notebook demonstrates metric computation, simple reporting, and optional MLflow logging for the Measurement Module.

In [1]:
import numpy as np
import pandas as pd
from fairness_pipeline_dev_toolkit.metrics import FairnessAnalyzer
from fairness_pipeline_dev_toolkit.integration.reporting import to_markdown_report

print("Versions:")
import fairness_pipeline_dev_toolkit as fpt
print("fairness_pipeline_dev_toolkit:", getattr(fpt, "__version__", "dev"))
import sys
print("python:", sys.version)


Versions:
fairness_pipeline_dev_toolkit: 0.1.0
python: 3.12.5 (main, Aug  6 2024, 19:08:49) [Clang 15.0.0 (clang-1500.3.9.4)]


## Load Sample Data
You can adjust this path to point at a real dataset later.

In [2]:
import os
path = "tests/fixtures/e2e_sample.csv" if os.path.exists("tests/fixtures/e2e_sample.csv") else "dev_sample.csv"
df = pd.read_csv(path)
df.head()

,y_true,y_pred,group,score
0,1,1,A,0.78
1,0,0,A,0.12
2,1,1,A,0.66
3,0,1,B,0.41
4,1,0,B,0.59


## Compute Metrics
- `demographic_parity_difference` (classification)
- `equalized_odds_difference` (classification)
- `mae_parity_difference` (regression-style using `score` vs `y_true`)

In [3]:
fa = FairnessAnalyzer(min_group_size=2, backend="native")
results = {}
results["demographic_parity_difference"] = fa.demographic_parity_difference(
    y_pred=df["y_pred"].to_numpy(), sensitive=df["group"].to_numpy()
)
results["equalized_odds_difference"] = fa.equalized_odds_difference(
    y_true=df["y_true"].to_numpy(), y_pred=df["y_pred"].to_numpy(), sensitive=df["group"].to_numpy()
)
results["mae_parity_difference"] = fa.mae_parity_difference(
    y_true=df["y_true"].to_numpy().astype(float),
    y_pred=df["score"].to_numpy().astype(float),
    sensitive=df["group"].to_numpy(),
)
results

{'demographic_parity_difference': Result(metric='demographic_parity_difference', value=0.19999999999999996, ci=None, effect_size=None, n_per_group={'A': 5, 'B': 5}),
 'equalized_odds_difference': Result(metric='equalized_odds_difference', value=0.6666666666666667, ci=None, effect_size=None, n_per_group={'A': 5, 'B': 5}),
 'mae_parity_difference': Result(metric='mae_parity_difference', value=0.16199999999999998, ci=None, effect_size=None, n_per_group={'A': 5, 'B': 5})}

## Markdown Report
A quick human-readable summary for PRs or artifacts.

In [4]:
md = to_markdown_report(results, title="Fairness Demo Report")
print(md)

# Optional: save
with open("artifacts_demo_report.md", "w", encoding="utf-8") as f:
    f.write(md)
print("Saved artifacts_demo_report.md")

# Fairness Demo Report

_Generated: 2025-10-30 13:23:54 UTC_

| Metric | Value | CI (95%) | Effect Size | n_per_group |
|---|---:|---|---:|---|
| `demographic_parity_difference` | 0.200000 | — | None | {"A": 5, "B": 5} |
| `equalized_odds_difference` | 0.666667 | — | None | {"A": 5, "B": 5} |
| `mae_parity_difference` | 0.162000 | — | None | {"A": 5, "B": 5} |

> Note: `—` indicates unavailable due to insufficient data or configuration.
Saved artifacts_demo_report.md
